In [ ]:
import ecole
import shutil
import os
import sys
import time
import numpy as np
from tqdm import trange
import pyscipopt as pyscip
import pickle as pkl

from generate_data import *
from utility import *

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
        
scip_parameters = {"branching/scorefunc": "s",
                   "separating/maxrounds": 0,
                   "limits/time": 360,
                   "conflict/enable": False,
                   "presolving/maxrounds": 0,
                   "presolving/maxrestarts": 0,
                   "separating/maxroundsroot" : 0,
                   "separating/maxcutsroot": 0,
                   "separating/maxcuts": 0,
                   "propagating/maxroundsroot" : 0,
                   "lp/presolving" : False,
                  }

generate_dataset(scip_parameters,path = "DataSet/",nb_cons = [1000],nb_var = [1],density = [0.1],100)

Generate with Row:1000,Col:1000,Density:0.100000


  1%|▍                                          | 1/100 [00:05<09:18,  5.64s/it]

pressed CTRL-C 1 times (5 times for forcing termination)


  2%|▊                                          | 2/100 [00:17<14:50,  9.09s/it]

pressed CTRL-C 1 times (5 times for forcing termination)


In [7]:
import ecole
import shutil
import os
import sys
import time
import numpy as np
import pyscipopt as pyscip
import pickle as pkl
from tqdm import trange

from utility import *
scip_parameters = {"branching/scorefunc": "s",
                   "separating/maxrounds": 0,
                   "limits/time": 360,
                   "conflict/enable": False,
                   "presolving/maxrounds": 0,
                   "presolving/maxrestarts": 0,
                   "separating/maxroundsroot" : 0,
                   "separating/maxcutsroot": 0,
                   "separating/maxcuts": 0,
                   "propagating/maxroundsroot" : 0,
                   "lp/presolving" : False,
                  }
def generate_dataset_CFL(scip_parameters,path = "DataSet_Evaluate_CFL/",\
                         n_customers = [500],n_facilities = [500],nb_instance = 100,only_problem = False):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.mkdir(path)

    env = ecole.environment.Branching(\
                observation_function = ecole.observation.NodeBipartite(),scip_params = scip_parameters)
    gapList = []
    for customers in n_customers:
        for facilities in n_facilities: 
            CFL = ecole.instance.CapacitatedFacilityLocationGenerator(n_customers = customers, n_facilities = facilities,continuous_assignment=False) 
            print("Generate with Row:%d,Col:%d" % (customers,facilities))
            for n in trange(1,nb_instance+1):
                done = False
                while(not done):
                    try:
                        problem_name = \
                            "CFL_{"+customers.__str__()+"*"+facilities.__str__()+"_"+n.__str__()+"}"
                        instance = next(CFL)
                        if only_problem:
                            #save problm lp
                            problem_name = \
                                "CFL_{"+customers.__str__()+"*"+facilities.__str__()+"_"+n.__str__()+"}"
                            instance.write_problem(path+problem_name+".lp")
                        else:
                            #save problm lp
                            problem_name = \
                                "CFL_{"+customers.__str__()+"*"+facilities.__str__()+"_"+n.__str__()+"}"
                            os.mkdir(path+problem_name)
                            instance.write_problem(path+problem_name+"/problem.lp")
                            #save features
                            obs, _, _, _, _ = env.reset(instance)
                            #save constraintes features
                            dumpRowFeatures(path+problem_name+"/constraints_features.json",obs.row_features)
                            #save variables features
                            dumpVariableFeatures(path+problem_name+"/variables_features.json",obs.variable_features)
                            #save edges features
                            original_indice = obs.variable_features[:,-1]
                            dumpEdgeFeatures(path+problem_name+"/edges_features.json",obs.edge_features,original_indice)
                            #get et save label
                            solver = ecole.scip.Model.from_file(path+problem_name+"/problem.lp")
                            aspyscip = solver.as_pyscipopt()
                            aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
                            aspyscip.optimize()
                            gapList.append(aspyscip.getGap())
                            dumpSolution_Ecole(path+problem_name+"/label.json",aspyscip)

                        done = True             
                    except Exception as ex:
#                             print("Erreur:%s"%ex)
                        done = False
                        shutil.rmtree(path+problem_name)
    gap = np.array(gapList)
    return np.mean(gap)

generate_dataset_CFL(scip_parameters,path = "DataSet_Evaluate_CFL/",\
                     n_customers = [30],n_facilities = [30,45],nb_instance = 40,only_problem = True)
    
    

Generate with Row:30,Col:30


100%|███████████████████████████████████████████| 40/40 [00:01<00:00, 31.58it/s]


Generate with Row:30,Col:45


100%|███████████████████████████████████████████| 40/40 [00:01<00:00, 21.63it/s]
/home/leonaire/anaconda3/envs/ecole/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/leonaire/anaconda3/envs/ecole/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan